In [1]:
print 'load libraries...'

load libraries...


In [2]:
# Aim: 

In [3]:
# Add path in order to load utility functions
import sys
sys.path.append(r"N:\18. Hot Spot\Custom Modules\Utils")
sys.path.append(r"N:\18. Hot Spot\Custom Modules\ETLMonthly")

In [4]:
# setup environment and import libraries
import os
import re
import datetime
import time
import getpass
import pandas as pd
import numpy as np
import networkx as nx
from datetime import datetime, date
from datetime import timedelta
import calendar as cd
from collections import OrderedDict
from collections import Counter
path=r'C:\ProgramData\Anaconda2\instantclient_12_2'
os.environ['PATH']=path
import cx_Oracle
from sqlalchemy import create_engine
import file_utils
import db_utils
import config
#set working directory
os.chdir(config.working_dir)

In [5]:
# db credentials authentication

In [6]:
print 'connect to database...'
conn_string = db_utils.db_connect(config.db_username,config.db_pwd)

connect to database...
Testing Connection...
Connection Succesful!


In [7]:
mtch = re.match('.*@', conn_string)
un, pwd = mtch.group().rstrip('@').split('/')

In [8]:
con = cx_Oracle.connect(conn_string)
cs_retail_loan_cust_data = pd.read_sql_query('select * from cs_retail_loan_cust', con)
con.close()

In [14]:
# get transaction relationship related to only the retail_loan guys as either sender or beneficiary

for i, f in enumerate(file_utils.get_files(r"N:\10. Executive\Original\ETL\trxn_ntwrk\monthly")):
    print i+1, '. ', f
    cur_ntwrk_df = file_utils.unzip(f)
        
    cur_ntwrk_merged_df = cur_ntwrk_df.merge(
                    cs_retail_loan_cust_data[['BRA_CODE','CUS_NUM']], how='left',
                    left_on=['SENDER_BRA_CODE','SENDER_CUS_NUM'],
                    right_on=['BRA_CODE','CUS_NUM']).merge(
                    cs_retail_loan_cust_data[['BRA_CODE','CUS_NUM']], how='left',
                    left_on=['BENEF_BRA_CODE','BENEF_CUS_NUM'],
                    right_on=['BRA_CODE','CUS_NUM'])
    
    cur_cs_retail_ntwrk = cur_ntwrk_merged_df[
                            (~cur_ntwrk_merged_df.BRA_CODE_x.isnull()) | 
                            (~cur_ntwrk_merged_df.BRA_CODE_y.isnull())].drop(
                            labels=['BRA_CODE_x','CUS_NUM_x','BRA_CODE_y','CUS_NUM_y'], axis=1)
    
    if_exists_var = 'append'
    if i==0:
        if_exists_var = 'replace'
    
    cur_cs_retail_ntwrk['REFERENCE_DATE']=pd.to_datetime(cur_cs_retail_ntwrk['REFERENCE_DATE'], format="%d%b%Y")
    cur_cs_retail_ntwrk['CTRL_DATE']=pd.to_datetime(cur_cs_retail_ntwrk['CTRL_DATE'], format="%d%b%Y")
    
    conn = create_engine('oracle+cx_oracle://{}:{}@dwdrscan.gtbank.com:1521/?service_name=odidb'.format(un, pwd))    
    cur_cs_retail_ntwrk.to_sql('cs_retail_ntwrk', conn, index=False, if_exists=if_exists_var, chunksize=10000,
                              dtype=db_utils.sqlcol(cur_cs_retail_ntwrk))
    

1 .  N:\10. Executive\Original\ETL\trxn_ntwrk\monthly\06Mar2020.csv.gz
2 .  N:\10. Executive\Original\ETL\trxn_ntwrk\monthly\28Feb2018.csv.gz
3 .  N:\10. Executive\Original\ETL\trxn_ntwrk\monthly\28Feb2019.csv.gz
4 .  N:\10. Executive\Original\ETL\trxn_ntwrk\monthly\29Feb2020.csv.gz
5 .  N:\10. Executive\Original\ETL\trxn_ntwrk\monthly\30Apr2018.csv.gz
6 .  N:\10. Executive\Original\ETL\trxn_ntwrk\monthly\30Apr2019.csv.gz
7 .  N:\10. Executive\Original\ETL\trxn_ntwrk\monthly\30Jun2018.csv.gz
8 .  N:\10. Executive\Original\ETL\trxn_ntwrk\monthly\30Jun2019.csv.gz
9 .  N:\10. Executive\Original\ETL\trxn_ntwrk\monthly\30Nov2018.csv.gz
10 .  N:\10. Executive\Original\ETL\trxn_ntwrk\monthly\30Nov2019.csv.gz
11 .  N:\10. Executive\Original\ETL\trxn_ntwrk\monthly\30Sep2018.csv.gz
12 .  N:\10. Executive\Original\ETL\trxn_ntwrk\monthly\30Sep2019.csv.gz
13 .  N:\10. Executive\Original\ETL\trxn_ntwrk\monthly\31Aug2018.csv.gz
14 .  N:\10. Executive\Original\ETL\trxn_ntwrk\monthly\31Aug2019.csv.gz
1

In [15]:
# limit trxn_ntwrk to the last 12months

con = cx_Oracle.connect(conn_string)
cs_retail_ntwrk = pd.read_sql_query('''
select b.sender_bra_code, b.sender_cus_num, b.benef_bra_code, b.benef_cus_num,
sum(tra_count) tra_count, sum(tra_amt) tra_amt,
b.sender_bra_code||'/'||b.sender_cus_num as sender_cust_id,
b.benef_bra_code||'/'||b.benef_cus_num benef_cust_id
from
cs_retail_loan_cust a
inner join cs_retail_ntwrk b
on (a.bra_code=b.sender_bra_code and a.cus_num=b.sender_cus_num) or
(a.bra_code=b.benef_bra_code and a.cus_num=b.benef_cus_num)
where b.reference_date between trunc(add_months(a.date_open, -12),'month')
and trunc(add_months(a.date_open, -1),'month')
group by b.sender_bra_code, b.sender_cus_num, b.benef_bra_code, b.benef_cus_num
''', con)
con.close()

In [16]:
start2=time.time()
G=nx.from_pandas_dataframe(cs_retail_ntwrk,'SENDER_CUST_ID','BENEF_CUST_ID',['TRA_COUNT','TRA_AMT'],create_using=nx.DiGraph())
print 'time elapsed for processsing : {} mins'.format((time.time()-start2)/60)

time elapsed for processsing : 1.48383333286 mins


In [17]:
cs_trxn_ntwrk_df = \
    pd.DataFrame(G.out_degree().items(), columns=['NODES','NTWRK_OUT_DEGREE']).merge(
    pd.DataFrame(G.out_degree(weight='TRA_COUNT').items(), columns=['NODES','NTWRK_OUT_TRA_COUNT']),
    on='NODES', how='outer').merge(
    pd.DataFrame(G.out_degree(weight='TRA_AMOUNT').items(), columns=['NODES','NTWRK_OUT_TRA_AMOUNT']),
    on='NODES', how='outer').merge(
    pd.DataFrame(G.in_degree().items(), columns=['NODES','NTWRK_IN_DEGREE']),
    on='NODES', how='outer').merge(
    pd.DataFrame(G.in_degree(weight='TRA_COUNT').items(), columns=['NODES','NTWRK_IN_TRA_COUNT']),
    on='NODES', how='outer').merge(           
    pd.DataFrame(G.in_degree(weight='TRA_AMOUNT').items(), columns=['NODES','NTWRK_IN_TRA_AMOUNT']),
    on='NODES', how='outer').merge(           
    pd.DataFrame(G.degree().items(), columns=['NODES','NTWRK_DEGREE']),
    on='NODES', how='outer').merge(           
    pd.DataFrame(G.degree(weight='TRA_COUNT').items(), columns=['NODES','NTWRK_TRA_COUNT']),
    on='NODES', how='outer').merge(
    pd.DataFrame(G.degree(weight='TRA_AMOUNT').items(), columns=['NODES','NTWRK_TRA_AMOUNT']),
    on='NODES', how='outer')

In [18]:
cs_trxn_ntwrk_df[['BRA_CODE','CUS_NUM']] = cs_trxn_ntwrk_df.NODES.str.split('/', expand=True).apply(pd.to_numeric, axis=1)

In [19]:
# cs_trxn_ntwrk_df.drop(labels='NODES', axis=1, errors='ignore', inplace=True)

In [20]:
# Has bad friends

In [21]:
list_of_bad_nodes = cs_retail_loan_cust_data[cs_retail_loan_cust_data['IS_BAD']==1].BRA_CODE.map(str) + '/' + \
cs_retail_loan_cust_data[cs_retail_loan_cust_data['IS_BAD']==1].CUS_NUM.map(str)

In [22]:
cs_retail_loan_cust_data['NODES']=cs_retail_loan_cust_data['BRA_CODE'].map(str)+'/'+cs_retail_loan_cust_data['CUS_NUM'].map(str)

In [23]:
for i, r in cs_retail_loan_cust_data.iterrows():
    try:
        neighbors = G.neighbors(r.NODES)
        nbr_bad_neighbors = pd.Series(neighbors).isin(list_of_bad_nodes).sum()
        cs_retail_loan_cust_data.loc[i,'NBR_BAD_NEIGHBORS'] = nbr_bad_neighbors
    except Exception as e:
        pass
        

In [24]:
cs_retail_loan_cust_data['NBR_BAD_NEIGHBORS'].fillna(0, inplace=True)

In [25]:
# get nodes that carried out bidirectional trxn

In [26]:
def get_bidirectional_nodes(G):
    bidirect_duplicated_edges = [(u,v) for (u,v) in G.edges()] + [(v,u) for (u,v) in G.edges()]
    bidirect_duplicated_edges_counter = Counter(bidirect_duplicated_edges)
    bidirect_duplicated_edges_items = list(set([i for (k,v) in bidirect_duplicated_edges_counter.items() if v>1 for i in k]))
    return bidirect_duplicated_edges_items
    

In [27]:
bidirect_nodes_df = pd.DataFrame(get_bidirectional_nodes(G), columns=['NODES'])

In [28]:
len(bidirect_nodes_df)

400377

In [29]:
bidirect_nodes_df['IS_BIDIRECT']=1

In [30]:
bidirect_nodes_df[['BRA_CODE','CUS_NUM']] = bidirect_nodes_df.NODES.str.split('/', expand=True).apply(pd.to_numeric, axis=1)

In [31]:
bidirect_nodes_df.drop(labels='NODES', axis=1, errors='ignore', inplace=True)

In [32]:
# to file

In [33]:
cs_trxn_ntwrk_info_df = cs_retail_loan_cust_data[['BRA_CODE','CUS_NUM','NBR_BAD_NEIGHBORS']].merge(cs_trxn_ntwrk_df,
               how='left', on=['BRA_CODE','CUS_NUM']).merge(bidirect_nodes_df,
               how='left', on=['BRA_CODE','CUS_NUM']).fillna(0)

In [34]:
cs_trxn_ntwrk_info_df.drop(labels='NODES', axis=1, inplace=True)

In [35]:
cs_trxn_ntwrk_info_df.to_csv('cs_trxn_ntwrk_info.csv', index=False)

In [36]:
cs_trxn_ntwrk_info_df

,BRA_CODE,CUS_NUM,NBR_BAD_NEIGHBORS,NTWRK_OUT_DEGREE,NTWRK_OUT_TRA_COUNT,NTWRK_OUT_TRA_AMOUNT,NTWRK_IN_DEGREE,NTWRK_IN_TRA_COUNT,NTWRK_IN_TRA_AMOUNT,NTWRK_DEGREE,NTWRK_TRA_COUNT,NTWRK_TRA_AMOUNT,IS_BIDIRECT
0,201,121934,0.0,12.0,25.0,12.0,5.0,12.0,5.0,17.0,37.0,17.0,1.0
1,201,122146,0.0,8.0,11.0,8.0,11.0,32.0,11.0,19.0,43.0,19.0,1.0
2,201,122155,0.0,12.0,92.0,12.0,8.0,86.0,8.0,20.0,178.0,20.0,1.0
3,201,122232,0.0,49.0,135.0,49.0,15.0,37.0,15.0,64.0,172.0,64.0,1.0
4,201,122234,0.0,52.0,170.0,52.0,15.0,30.0,15.0,67.0,200.0,67.0,1.0
5,201,122250,0.0,42.0,141.0,42.0,5.0,6.0,5.0,47.0,147.0,47.0,0.0
6,201,122275,0.0,122.0,798.0,122.0,11.0,46.0,11.0,133.0,844.0,133.0,1.0
7,201,122448,0.0,20.0,116.0,20.0,10.0,60.0,10.0,30.0,176.0,30.0,1.0
8,201,122526,0.0,18.0,38.0,18.0,34.0,80.0,34.0,52.0,118.0,52.0,1.0
9,201,122552,0.0,11.0,43.0,11.0,15.0,19.0,15.0,26.0,62.0,26.0,1.0
